# **Задача 4. Сортировка на GPU с использованием CUDA**

Diana Kim ADA-2403M

**Практическое задание (25 баллов)**

Реализуйте параллельную сортировку слиянием на GPU с использованием CUDA:
*   разделите массив на подмассивы, каждый из которых обрабатывается отдельным блоком;
*   выполните параллельное слияние отсортированных подмассивов;
*   замерьте производительность для массивов размером 10 000 и 100 000 элементов.


**Подключение GPU и CUDA**

In [1]:
!nvidia-smi

Tue Dec 30 08:22:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


## **Практические задания**

In [6]:
%%writefile Task4.cu

#include <cuda_runtime.h>                 // для функций CUDA, которые мне надо
#include <device_launch_parameters.h>               // подключает переменные threadIdx, blockIdx и параметры запуска ядра
#include <iostream>                       // ввод/вывод
#include <vector>                           // подключает vector
#include <random>                         // для генерации случайных чисел
#include <chrono>                          // для измерения времени выполнения
#include <algorithm>                      // для алгоритмов, которые мне надо

using namespace std;

#define CUDA_CHECK(call) do {                        /* делает проверку CUDA вызова */ \
    cudaError_t err = (call);                           /* сохраняет код ошибки */ \
    if (err != cudaSuccess) {                           /* проверяет, что ошибки нет */ \
        cerr << "CUDA error: " << cudaGetErrorString(err)                /* выводит текст ошибки */ \
             << " at " << __FILE__ << ":" << __LINE__ << "\n";                    /* выводит место ошибки */ \
        exit(1);                             /* завершает программу */ \
    }                                                        \
} while(0)

static const int CHUNK = 256;                     // задает размер подмассива для одного блока
static const int INF = 2147483647;            // задает большое число для заполнения хвоста


__global__ void sortChunksBitonic(const int* in, int* out, int n) {           // сортирует каждый CHUNK параллельно по блокам
    __shared__ int s[CHUNK];                    // создает shared память для быстрого доступа

    int tid = threadIdx.x;                      // получает номер потока в блоке
    int base = blockIdx.x * CHUNK;                      // вычисляет начало куска
    int idx = base + tid;                            // вычисляет глобальный индекс

    s[tid] = (idx < n) ? in[idx] : INF;                       // загружает данные или INF
    __syncthreads();                     // синхронизирует потоки после загрузки

    for (int k = 2; k <= CHUNK; k <<= 1) {                 // увеличивает размер сортируемых блоков
        for (int j = k >> 1; j > 0; j >>= 1) {              // задает расстояние сравнения
            int ixj = tid ^ j;                  // вычисляет партнера через XOR
            if (ixj > tid) {              // делает сравнение один раз на пару
                bool asc = ((tid & k) == 0);           // определяет направление сортировки
                int a = s[tid];             // берет элемент a
                int b = s[ixj];                // берет элемент b
                if (asc) {                // сортирует по возрастанию
                    if (a > b) { s[tid] = b; s[ixj] = a; }                   // меняет местами при необходимости
                } else {                       // сортирует по убыванию
                    if (a < b) { s[tid] = b; s[ixj] = a; }               // меняет местами при необходимости
                }
            }
            __syncthreads();              // синхронизирует потоки после шага
        }
    }

    if (idx < n) out[idx] = s[tid];             // записывает результат обратно в память
}


__global__ void mergePass(const int* input, int* output, int n, int width) {                    // сливает пары отсортированных частей
    int pairIdx = blockIdx.x;                  // получает номер пары
    int start = pairIdx * (2 * width);                 // вычисляет начало пары
    if (start >= n) return;                     // выходит, если за границей

    int mid = (start + width < n) ? (start + width) : n;          // вычисляет середину
    int end = (start + 2 * width < n) ? (start + 2 * width) : n;                // вычисляет конец

    if (threadIdx.x == 0) {                 // делает слияние одним потоком
        int i = start;                         // задает индекс левой части
        int j = mid;                    // задает индекс правой части
        int k = start;                    // задает индекс записи

        while (i < mid && j < end) {                      // выполняет слияние пока есть элементы
            if (input[i] <= input[j]) output[k++] = input[i++];           // записывает меньший элемент
            else                      output[k++] = input[j++];                 // записывает другой элемент
        }
        while (i < mid) output[k++] = input[i++];          // дописывает остаток слева
        while (j < end) output[k++] = input[j++];            // дописывает остаток справа
    }
}


float gpuMergeSort(int* d_in, int* d_tmp, int n) {             // запускает сортировку на GPU и возвращает время
    cudaEvent_t st, en;                        // создает события CUDA для таймера
    CUDA_CHECK(cudaEventCreate(&st));              // создает событие старта
    CUDA_CHECK(cudaEventCreate(&en));              // создает событие конца
    CUDA_CHECK(cudaEventRecord(st));                // запускает таймер

    int blocks = (n + CHUNK - 1) / CHUNK;              // вычисляет количество блоков для чанков

    sortChunksBitonic<<<blocks, CHUNK>>>(d_in, d_tmp, n);                // сортирует чанки параллельно по блокам
    CUDA_CHECK(cudaGetLastError());                  // проверяет запуск ядра
    CUDA_CHECK(cudaDeviceSynchronize());             // ждет завершения сортировки чанков

    int* src = d_tmp;                 // задает источник как временный массив
    int* dst = d_in;                    // задает приемник как исходный массив

    for (int width = CHUNK; width < n; width *= 2) {            // увеличивает размер отсортированных прогонов
        int pairs = (n + (2 * width) - 1) / (2 * width);                 // вычисляет количество пар для слияния
        mergePass<<<pairs, 256>>>(src, dst, n, width);                  // выполняет слияние пар прогонов
        CUDA_CHECK(cudaGetLastError());                 // проверяет запуск ядра
        CUDA_CHECK(cudaDeviceSynchronize());                  // ждет завершения слияния
        std::swap(src, dst);                // меняет местами src и dst
    }

    if (src != d_in) {                                          // проверяет, где лежит итог
        CUDA_CHECK(cudaMemcpy(d_in, src, n * sizeof(int), cudaMemcpyDeviceToDevice));               // копирует итог в d_in
    }

    CUDA_CHECK(cudaEventRecord(en));                     // останавливает таймер
    CUDA_CHECK(cudaEventSynchronize(en));                     // ждет окончания события
    float ms = 0.0f;                               // создает переменную времени
    CUDA_CHECK(cudaEventElapsedTime(&ms, st, en));           // получает время в миллисекундах
    CUDA_CHECK(cudaEventDestroy(st));                  // удаляет событие старта
    CUDA_CHECK(cudaEventDestroy(en));              // удаляет событие конца

    return ms;          // возвращает время
}


bool isSorted(const vector<int>& a) {                // проверяет, отсортирован ли массив
    return std::is_sorted(a.begin(), a.end());              // возвращает результат проверки
}



void runOne(int n) {                           // тест для конкретного размера
    mt19937 rng(123);                             // задает генератор с фиксированным seed
    uniform_int_distribution<int> dist(1, 1000000);             // задает диапазон чисел

    vector<int> h(n);                     // создает массив на CPU
    for (int i = 0; i < n; i++) h[i] = dist(rng);                  // заполняет массив случайными числами

    int* d_in = nullptr;                    // создаёт указатель на вход GPU
    int* d_tmp = nullptr;                      // создаёт указатель на временный GPU
    CUDA_CHECK(cudaMalloc(&d_in, n * sizeof(int)));           // выделяет память на GPU
    CUDA_CHECK(cudaMalloc(&d_tmp, n * sizeof(int)));            // выделяет память на GPU

    CUDA_CHECK(cudaMemcpy(d_in, h.data(), n * sizeof(int), cudaMemcpyHostToDevice));             // копирует данные на GPU

    float gpu_ms = gpuMergeSort(d_in, d_tmp, n);       // выполняет сортировку и получает время

    vector<int> out(n);                     // создаёт массив результата на CPU
    CUDA_CHECK(cudaMemcpy(out.data(), d_in, n * sizeof(int), cudaMemcpyDeviceToHost));          // копирует результат на CPU

    cout << "\n N = " << n << " \n";            // выводит размер массива
    cout << "GPU merge time: " << gpu_ms << " ms\n";            // выводит время GPU сортировки
    cout << "sorted: " << (isSorted(out) ? "yes" : "no") << "\n";          // выводит корректность

    CUDA_CHECK(cudaFree(d_in));                  // освобождает память d_in
    CUDA_CHECK(cudaFree(d_tmp));                  // освобождает память d_tmp
}

int main() {
    CUDA_CHECK(cudaFree(0));                    // делает прогрев CUDA
    runOne(10000);                           // запускает тест для 10000
    runOne(100000);                    // запускает тест для 100000
    return 0;                          // завершает программу
}

Overwriting Task4.cu


In [7]:
!nvcc Task4.cu -O2 -o Task4 -gencode arch=compute_75,code=sm_75
!./Task4


 N = 10000 
GPU merge time: 5.2024 ms
sorted: yes

 N = 100000 
GPU merge time: 44.6086 ms
sorted: yes
